In [1]:
from collections import defaultdict

import cv2
import numpy as np
from ultralytics import YOLO

In [2]:
# Load the YOLO11 model
model = YOLO("yolo11l.pt")

# Open the video file
video_path = "samples/vietnam.mp4"
cap = cv2.VideoCapture(video_path)

100%|██████████| 49.0M/49.0M [00:02<00:00, 20.7MB/s]


In [3]:
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Create VideoWriter object
video_name = video_path.split("/")[-1]
output_path = f"run/{video_name.split('.')[0]}_tracked.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

In [4]:
# Store the track history
track_history = defaultdict(lambda: [])

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLO11 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, show=False)

        # Get the boxes and track IDs (with error handling)
        boxes = results[0].boxes.xywh.cpu()
        try:
            track_ids = results[0].boxes.id
            if track_ids is not None:
                track_ids = track_ids.int().cpu().tolist()
            else:
                track_ids = []  # No tracks found in this frame
        except AttributeError:
            track_ids = []  # Handle case where tracking fails

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks only if we have valid tracking data
        if track_ids:
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point

                if len(track) > 120:  # retain 30 tracks for 30 frames
                    track.pop(0)

                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(
                    annotated_frame,
                    [points],
                    isClosed=False,
                    color=(230, 230, 230),
                    thickness=4,
                )
        # Write the frame to output video
        out.write(annotated_frame)
    else:
        # Break the loop if the end of the video is reached
        break

# Release everything
cap.release()
out.release()
print(f"Video has been saved to {output_path}")

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.0 MB/s eta 0:00:00 0:00:01

requirements: AutoUpdate success ✅ 2.0s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 384x640 19 persons, 6 cars, 26 motorcycles, 2 buss, 2 trucks, 47.5ms
Speed: 3.3ms preprocess, 47.5ms inference, 258.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 6 cars, 26 motorcycles, 1 bus, 2 trucks, 13.2ms
Speed: 1.2ms preprocess, 13.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 5 cars, 25 motorcycles, 1 bus, 3 trucks, 13.1ms
Speed: 1.1ms preprocess, 13.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 5 cars, 25 motorcycles, 1 bus, 4 trucks, 13.1ms
Speed: 1.1ms preprocess, 13.1ms inference, 0.7ms postprocess per image at shape (1, 3,